# Multiple inheritance in Python: Even more to learn

I felt so proud of myself when I first learned how to reason about multiple inheritance in Python, and the MRO in particular.  The _method resolution order_ (MRO) is the order in which Python arranges name lookups in a class hierarchy.  For single inheritance, this is simple and obvious, and works like most languages with object-oriented features.

However, for multiple inheritance, thing become a little trickier. Here is an example:

In [6]:
class A:
    def __init__(self):
        print('A: before init')
        super().__init__()
        print('A: after init')

class B(A):
    def __init__(self):
        print('B: before init')
        super().__init__()
        print('B: after init')
        
class C:
    def __init__(self):
        print('C: before init')
        super().__init__()
        print('C: after init')
        
class D(B, C):
    def __init__(self):
        print('D: before init')
        super().__init__()
        print('D: after init')

Here we have four classes, `A` through `D`. The important stuff begins at `D`.  Do you see how `D` inherits from both `B` _and_ `C`?  If you have not already learned how Python deals with this, then you're in for a treat:

In [8]:
d = D()

D: before init
B: before init
A: before init
C: before init
C: after init
A: after init
B: after init
D: after init


Making an instance of `D`, we see the cascade of `print()` calls.  Here, it is obvious that Python has arranged the sequence of `super()` calls such that lookup proceeds **depth-first**.  In other words, the lookup being triggered by `super()` will proceed through the entire superclass list of `B`, before working through the same for `C`.

_Side-note_: If `A` did **not** also make a call to `super()` in its initializer, the chain would have been broken and `C.__init__()` would **not** have been called.  This means that for classes to cooperate in a multiple-inheritance arrangement, even base classes _must_ call `super()`.

But I digress.  Let's get back to the MRO.  It's easy to see the MRO being used:

for _ in D.__mro__: print(_)

This makes the order obvious.  And this is what I learned a few years ago.  However, a [recent talk](https://youtu.be/l8u8VENJhpM) by Mike Leonard, _The Wizardry of Metaprogramming_, had an example of multiple inheritance in which it seemed like the MRO was constructed in a **breadth-first** sequence, the exact opposite of how I thought it worked! Here is a slightly reworked example:

In [12]:
class A:
    def __init__(self):
        print('A: before init')
        super().__init__()
        print('A: after init')

class B(A):
    def __init__(self):
        print('B: before init')
        super().__init__()
        print('B: after init')
        
class C(A):
    def __init__(self):
        print('C: before init')
        super().__init__()
        print('C: after init')
        
class D(B, C):
    def __init__(self):
        print('D: before init')
        super().__init__()
        print('D: after init')

This is _almost_ the same as the previous code, but I've made one tiny change. Can you spot it?  I will reveal in a moment, but first, let's create an instance of `D` and print out the MRO:

In [13]:
d = D()

D: before init
B: before init
C: before init
A: before init
A: after init
C: after init
B: after init
D: after init


In [14]:
for _ in D.__mro__: print(_)

<class '__main__.D'>
<class '__main__.B'>
<class '__main__.C'>
<class '__main__.A'>
<class 'object'>


Whaaaat?  Now the sequence seems to indicate that `C` immediately follows `B`, and the root class `A` comes last!  That is no longer "depth-first".

It turns out that Python does things a little differently when a class appears **multiple times** in the MRO.  In this situation, all duplicate appearances (of `A` in this case) will be removed **except for the last**.  Earlier, I had changed the code for the `C` class to also inherit from `A`.  This meant that both `B` **and** `C` inherit from `A`, and a naive depth-first construction of the MRO results in two entries for `A`. Python therefore deletes all appearances except the last.

This is all documented in tremendous detail in [The Python 2.3 Method Resolution Order](https://www.python.org/download/releases/2.3/mro/).  I probably should have read it a long time ago.